In the first step, we are installing the necessary libraries used in this notebook


In [1]:
import numpy as np 
!pip install geopy
!pip install folium
import geopy
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

print('Libraries imported.')


Libraries imported.


Inorder to scrape the Wikipedia website, we are going to use Beautifulsoup python package.Before doing that, we should install the beautifulsoup & html parsers packages into this notebook

In [2]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests


In the below lines of code,With the help of BeautifulSoup ,the toronto city neighborhood table is extracted and stored in a csv file called 'Wiki_scrape.csv'

In [3]:
from bs4 import BeautifulSoup
import csv

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup= BeautifulSoup(source,'lxml')

csv_file= open('Wiki_scrape.csv','w')

csv_writer= csv.writer(csv_file)
csv_writer.writerow(['Postcode','Borough','Neighborhood'])

tables = soup.find_all('table', class_='wikitable sortable')

for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode','Borough','Neighborhood']:
        break

with open('soup', 'w') as toronto_df:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        Postcode,Borough,Neighborhood = [td.text.strip() for td in tds[:3]]
        csv_writer.writerow([Postcode,Borough,Neighborhood])
        

Now using pandas library, the CSV file is read and stored in a pandas dataframe 'df'.But data wrangling is required for this dataframe inorder to proceed further.

In [4]:
df=pd.read_csv('Wiki_scrape.csv')
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In the below cell, we are dropping the rows where the value of Borough column is 'Not assigned'

In [5]:
df=df[df.Borough !='Not assigned']
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Now we are going to change'Not assigned' values in Neighborhood column with it's respective Borough.

In [6]:
df=df.astype('str')
df=df.replace('Not assigned','Queen\'s Park')
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


As instructed, we are grouping the dataset based on each Borough's Postal code. 

In [7]:
df = df.groupby('Postcode').agg({'Borough':'first','Neighborhood':', '.join}).reset_index()
df

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
df.rename(columns={"Postcode": "Postal Code"})


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
URL='https://cocl.us/Geospatial_data'
geo_df=pd.read_csv(URL)
geo_df=geo_df.astype('str')
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686299999996,-79.19435340000001
1,M1C,43.7845351,-79.16049709999999
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.21691740000001
4,M1H,43.773136,-79.23947609999999
5,M1J,43.7447342,-79.23947609999999
6,M1K,43.7279292,-79.26202940000002
7,M1L,43.711111700000004,-79.2845772
8,M1M,43.716316,-79.23947609999999
9,M1N,43.692657000000004,-79.2648481


In [10]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Toronto_df = pd.read_csv(body)
Toronto_df


,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude
0,0.0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1.0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2.0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3.0,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4.0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,5.0,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,6.0,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,7.0,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,8.0,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,9.0,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
